In [1]:
from sim import Team, GameState, Player
import yaml
import matplotlib.pyplot as plt

c:\Github\nfl_sim\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
with open("models/feature_config.yaml", "r") as file:
    config = yaml.safe_load(file)

vik = Team("LV", 2024, 14)
det = Team("PHI", 2024, 13)
game = GameState(vik, det, config, **{"wind": 5, "temp": 90, "spread_line": -12.5})
game.play_game()
game.game_results(df=True).groupby(["team_name"]).sum()[
    ["receiving_yards", "rushing_yards"]
]


c:\Github\nfl_sim\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\data\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


PHI has won 38 - 21


,receiving_yards,rushing_yards
team_name,,
LV,257,137
PHI,367,185


In [3]:
df = (
    game.game_results(df=True)
    .groupby(["team_name"])
    .sum()[["receiving_yards", "rushing_yards", "carries"]]
)
df["ypc"] = df["rushing_yards"] / df["carries"]
df

,receiving_yards,rushing_yards,carries,ypc
team_name,,,,
LV,257,137,31,4.419355
PHI,367,185,43,4.302326


In [4]:
def run_sim(team1, team2, num_sims, config=config):
    results = []
    scores = []
    vik = Team(team1, 2024, 13)
    det = Team(team2, 2024, 13)
    game = GameState(vik, det, config, **{"wind": 5, "temp": 60, "spread_line": 6.5})
    for i in range(num_sims):
        game.start_game()
        game.play_game()
        scores.append((game.home.score, game.away.score))
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results, scores


results, scores = run_sim(
    "CAR", "PHI", 10
)  # 28.8 -> 22.8 by pulling game initialization out of loop

CAR has won 42 - 20
QB -1 CAR


ValueError: You want a player that does not exist

In [ ]:
df = pd.json_normalize(results)
df.groupby(["name", "id"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].agg(["median", "mean"]).sort_values(by=[("receiving_yards", "mean")], ascending=False)

receiving_yards        targets       receptions  \
                                      median   mean  median  mean     median   
name              id                                                           
A.J. Brown        00-0035676            61.0  60.75     7.0  7.31        4.0   
David Moore       00-0033589            33.0  39.13     5.0  4.86        3.0   
Xavier Legette    00-0039342            37.5  38.85     4.0  4.92        3.0   
Jalen Coker       00-0039491            27.5  33.72     4.0  4.16        2.0   
Dallas Goedert    00-0034351            30.5  33.38     4.0  4.45        3.0   
Chuba Hubbard     00-0036555            17.5  22.66     3.0  2.75        2.0   
Jahan Dotson      00-0037741            10.0  17.54     2.0  2.72        2.0   
Saquon Barkley    00-0034844             8.0  15.49     2.0  2.77        2.0   
Tommy Tremble     00-0037005             7.0  12.81     2.0  2.14        1.0   
Ja'Tavion Sanders 00-0039356             6.0  12.67     2.0  1.82        1.0   
Grant Calcaterra  00-0037086             9.0  12.05     2.0  2.24        1.0   
Kenneth Gainwell  00-0036919             0.0   7.52     1.0  1.08        0.0   
Britain Covey     00-0037132             0.0   6.89     1.0  1.20        1.0   
Johnny Wilson     00-0039236             0.0   6.40     1.0  0.95        0.0   
Feleipe Franks    00-0036825             0.0   3.46     0.0  0.53        0.0   
Raheem Blackshear 00-0037429             0.0   2.94     0.0  0.37        0.0   
Ainias Smith      00-0039747             0.0   1.04     0.0  0.07        0.0   
C.J. Uzomah       00-0032134             0.0   0.03     0.0  0.03        0.0   
Bryce Young       00-0039150             0.0   0.00     0.0  0.00        0.0   
Jalen Hurts       00-0036389             0.0   0.00     0.0  0.00        0.0   
Kenny Pickett     00-0038102             0.0   0.00     0.0  0.00        0.0   
Andy Dalton       00-0027973             0.0   0.00     0.0  0.00        0.0   
Will Shipley      00-0039746             0.0   0.00     0.0  0.00        0.0   

                                   air_yards        completions         \
                              mean    median   mean      median   mean   
name              id                                                     
A.J. Brown        00-0035676  4.57      32.5  37.05         0.0   0.00   
David Moore       00-0033589  2.67      18.5  23.64         0.0   0.00   
Xavier Legette    00-0039342  2.86      20.0  22.40         0.0   0.00   
Jalen Coker       00-0039491  2.30      16.0  22.06         0.0   0.00   
Dallas Goedert    00-0034351  2.90      13.0  17.73         0.0   0.00   
Chuba Hubbard     00-0036555  1.91       1.0   5.70         0.0   0.00   
Jahan Dotson      00-0037741  1.77       3.5   7.38         0.0   0.00   
Saquon Barkley    00-0034844  2.05       0.0   3.94         0.0   0.00   
Tommy Tremble     00-0037005  1.14       0.5   4.15         0.0   0.00   
Ja'Tavion Sanders 00-0039356  1.03       1.0   5.95         0.0   0.00   
Grant Calcaterra  00-0037086  1.42       2.0   4.85         0.0   0.00   
Kenneth Gainwell  00-0036919  0.70       0.0   1.93         0.0   0.00   
Britain Covey     00-0037132  0.76       0.0   2.69         0.0   0.00   
Johnny Wilson     00-0039236  0.51       0.0   2.35         0.0   0.00   
Feleipe Franks    00-0036825  0.39       0.0   1.12         0.0   0.00   
Raheem Blackshear 00-0037429  0.25       0.0   1.08         0.0   0.00   
Ainias Smith      00-0039747  0.06       0.0   0.02         0.0   0.00   
C.J. Uzomah       00-0032134  0.01       0.0   0.03         0.0   0.00   
Bryce Young       00-0039150  0.00       0.0   0.00         0.0   0.00   
Jalen Hurts       00-0036389  0.00       0.0   0.00        15.0  14.75   
Kenny Pickett     00-0038102  0.00       0.0   0.00         0.0   0.00   
Andy Dalton       00-0027973  0.00       0.0   0.00        13.0  12.55   
Will Shipley      00-0039746  0.00       0.0   0.00         0.0   0.00   

                       

In [ ]:
df.groupby(["team_name", "game_number"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].sum().groupby(["team_name"]).mean().sort_values(
    by=["receiving_yards"], ascending=False
)

,receiving_yards,targets,receptions,air_yards,completions,attempts,rushing_yards,carries
team_name,,,,,,,,
LV,311.18,40.76,26.54,175.96,26.54,40.76,131.48,27.77
KC,288.14,38.74,25.80,145.67,25.80,38.74,115.59,25.45


In [ ]:
sched = pd.read_csv("week_13.csv", index_col=0)
sched.iloc[11]

game_id           2024_13_TB_CAR
season                      2024
week                          13
home_team                    CAR
away_team                     TB
home_moneyline             205.0
spread_line                 -6.5
total_line                  46.5
temp                        50.0
wind                         8.0
Name: 6896, dtype: object

In [ ]:
def run_sim(num_sims, sched, config=config):
    results = []
    scores = []
    for i in range(num_sims):
        vik = Team(sched["home_team"], sched["season"], sched["week"])
        det = Team(sched["away_team"], sched["season"], sched["week"])
        game = GameState(vik, det, config, **sched)
        game.start_game()
        game.play_game()
        scores.append((game.home.score, game.away.score))
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results, scores


# results, scores = run_sim('KC','LV',100)

In [ ]:
results = []
test = True
if test:
    for t in [11]:
        # for t in range(len(sched)):
        game1 = sched.iloc[t].to_dict()
        try:
            results, scores = run_sim("KC", "LV", 100, game1)
        except Exception as _:
            print("oops")

CAR has won 13 - 7
CAR has won 31 - 7
CAR has won 30 - 6
CAR has won 26 - 17
TB has won 28 - 24
CAR has won 34 - 27
TB has won 24 - 14
TB has won 27 - 20
TB has won 28 - 24
CAR has won 24 - 20
CAR has won 17 - 13
CAR has won 21 - 13
CAR has won 40 - 14
CAR has won 24 - 14
TB has won 16 - 16
TB has won 20 - 19
TB has won 23 - 21
CAR has won 26 - 17
TB has won 21 - 17
TB has won 26 - 17
TB has won 17 - 17
CAR has won 13 - 10
CAR has won 24 - 9
CAR has won 45 - 17
CAR has won 44 - 17
TB has won 31 - 23
CAR has won 31 - 21
TB has won 20 - 7
TB has won 17 - 17
CAR has won 23 - 17
TB has won 27 - 6
CAR has won 28 - 27
TB has won 21 - 16
TB has won 24 - 6
TB has won 24 - 14
CAR has won 41 - 16
CAR has won 23 - 17
CAR has won 41 - 38
TB has won 31 - 14
TB has won 20 - 17
TB has won 34 - 14
CAR has won 20 - 10
TB has won 10 - 10
TB has won 31 - 10
CAR has won 34 - 17
TB has won 24 - 23
TB has won 21 - 7
TB has won 26 - 26
CAR has won 41 - 16
CAR has won 20 - 9
TB has won 24 - 24
CAR has won 24 

In [ ]:
weekly = pd.read_csv("data/weekly.csv", index_col=0)
pit = weekly.loc[
    (weekly.week == 12)
    & (weekly.season == 2024)
    & ((weekly.recent_team == "PIT") | (weekly.recent_team == "CLE"))
]